In [12]:
# Complete working script for chatbot memory and context management

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationChain
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.schema import Document
from langchain_groq import ChatGroq
from langchain.chains.summarize import load_summarize_chain
from langchain_community.embeddings import HuggingFaceEmbeddings
import os
from dotenv import load_dotenv

load_dotenv()
groq_api_key = os.getenv("groq_api")

# -------------------- Setup LLM and Embeddings --------------------

llm = ChatGroq(
    model_name='gemma2-9b-it',
    temperature=0,  # Keep temperature at 0 for more factual/less creative answers
    groq_api_key=groq_api_key
)
embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# -------------------- Long-term Memory (FAISS Vector Store) --------------------

# Add user-specific long-term knowledge
doc_texts = [
    "User's name is Nishanthan and prefers analogies.",
    "User is preparing for AI/ML interviews.",
    "User has experience with RAG-based systems."
]
documents_for_faiss = [Document(page_content=text) for text in doc_texts]

# Create vector store
vectorstore = FAISS.from_documents(documents_for_faiss, embedding_model)




In [13]:
# -------------------- Short-term Memory with Summarization --------------------
short_term_memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=1000,
    return_messages=True
)

# -------------------- Conversation Chain --------------------
conversation = ConversationChain(
    llm=llm,
    memory=short_term_memory,
    verbose=True
)

In [14]:
# -------------------- Simulate User Conversation --------------------
def chat_with_bot(user_input, store_to_long_term=False):
    print("\nUser:", user_input)
    response = conversation.predict(input=user_input)
    print("Assistant:", response)

    if store_to_long_term:
        vectorstore.add_documents([
            Document(page_content=f"User: {user_input}"),
            Document(page_content=f"Assistant: {response}")
        ])

    print("\n--- Current Buffer Memory (Summarized + Recent) ---")
    print(short_term_memory.buffer)

In [15]:
# -------------------- Simulate User Conversation --------------------
user_inputs = [
    "Hi, I'm Nishanthan. Can you help me prepare for a data science interview?",
    "What are the key differences between supervised and unsupervised learning?",
    "Can you explain transfer learning in simple terms?"
]

for user_input in user_inputs:
    print("\nUser:", user_input)
    response = conversation.predict(input=user_input)
    print("Assistant:", response)



User: Hi, I'm Nishanthan. Can you help me prepare for a data science interview?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: Hi, I'm Nishanthan. Can you help me prepare for a data science interview?
AI:

> Finished chain.
Assistant: Hi Nishanthan, it's great to meet you! I'd love to help you prepare for your data science interview.  

To give you the best advice, tell me a little more about what you need help with. For example:

* **What kind of data science role are you interviewing for?** (e.g., Machine Learning Engineer, Data Analyst, Data Scientist)
* **What are you most nervous about?** (e.g., technical questions, behavioral questions, coding challenges)
* **Are there any specific topics you'd lik

In [ ]:

# -------------------- Retrieve Long-Term Memory --------------------
# query = "What do you know about the user?"
# retrieved_docs = vectorstore.similarity_search(query)
# print("\n--- Retrieved Long-term Memory ---")
# for doc in retrieved_docs:
#     print("-", doc.page_content)

# -------------------- Summarize Full Conversation --------------------
# summary_chain = load_summarize_chain(llm, chain_type="map_reduce")
# docs = [Document(page_content=msg.content) for msg in short_term_memory.chat_memory.messages]
# summary = summary_chain.run(docs)
# print("\n--- Conversation Summary ---")
# print(summary)



--- Retrieved Long-term Memory ---
- User is preparing for AI/ML interviews.
- User's name is Nishanthan and prefers analogies.
- User has experience with RAG-based systems.


C:\Users\nis44\AppData\Local\Temp\ipykernel_6212\3983167649.py:11: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summary = summary_chain.run(docs)



--- Conversation Summary ---
This text provides a brief overview of data science interview preparation and key machine learning concepts. 

It emphasizes the importance of tailoring preparation to the specific role and the interviewee's concerns. 

It then explains supervised and unsupervised learning, highlighting their differences in data usage and evaluation metrics. 

Finally, it introduces transfer learning as a technique for leveraging existing knowledge to accelerate learning new tasks in machine learning. 






In [16]:
# summary_chain = load_summarize_chain(llm, chain_type="map_reduce")
# docs = [Document(page_content=msg.content) for msg in short_term_memory.chat_memory.messages]
# summary = summary_chain.run(docs)
# print("\n--- Conversation Summary ---")
# print(summary)

# -------------------- Follow-up: Short-Term Only --------------------
chat_with_bot("Can you give me tips for solving ML case studies?")
chat_with_bot("What should I focus on when answering business case questions in interviews?")


User: Can you give me tips for solving ML case studies?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content="Hi, I'm Nishanthan. Can you help me prepare for a data science interview?", additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Nishanthan, it's great to meet you! I'd love to help you prepare for your data science interview.  \n\nTo give you the best advice, tell me a little more about what you need help with. For example:\n\n* **What kind of data science role are you interviewing for?** (e.g., Machine Learning Engineer, Data Analyst, Data Scientist)\n* **What are you most nervous about?** (e.g., technical questions, behavioral questions, coding challenges)\n* **Are there an